In [1]:
import sqlite3
import pandas as pd

In [154]:
con = sqlite3.connect(':memory:')

In [155]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [158]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')


In [159]:
data = pd.read_csv(f'Orders.csv', sep = ';')
data["OrderDate"]  = pd.to_datetime(data["OrderDate"], format="%d.%m.%Y")
data.to_sql("Orders", con, index=False, if_exists='replace')

196

In [160]:
data = pd.read_csv(f'Employees.csv', sep = ';')
data["BirthDate"]  = pd.to_datetime(data["BirthDate"], format="%d.%m.%Y")
data.to_sql("Employees", con, index=False, if_exists='replace')

10

In [15]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [6]:
pd.read_sql(
    '''
    SELECT *FROM Customers
    where city like "%mad%"
    ''',
    con,
)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,8,Bólido Comidas preparadas,Martín Sommer,"C/ Araquil, 67",Madrid,28023,Spain
1,22,FISSA Fabrica Inter. Salchichas S.A.,Diego Roel,"C/ Moralzarzal, 86",Madrid,28034,Spain
2,69,Romero y tomillo,Alejandra Camino,"Gran Vía, 1",Madrid,28001,Spain


In [7]:
pd.read_sql(
    '''
    SELECT CustomerName,Country, Address  FROM Customers
    where Country in ("Germany", "France") or city = "Madrid"
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [8]:
pd.read_sql(
    '''
    SELECT
    country,
    count(*) as order_count
    from (SELECT   Orders.*, Customers.Country FROM Orders
    join Customers on Customers.CustomerID = Orders.CustomerID ) count_order
    group by Country
    order by order_count DESC
    limit 3
    ''',
    con,
)

,Country,order_count
0,USA,29
1,Germany,25
2,Brazil,19


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [161]:
pd.read_sql(
    '''
    SELECT * FROM  (SELECT   Orders.*, Customers.CustomerName FROM Orders
    join Customers on Customers.CustomerID = Orders.ShipperID order by OrderDate DESC )
    ''',
    con,
)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,CustomerName
0,10443,66,8,1997-02-12 00:00:00,1,Alfreds Futterkiste
1,10442,20,3,1997-02-11 00:00:00,2,Ana Trujillo Emparedados y helados
2,10440,71,4,1997-02-10 00:00:00,2,Ana Trujillo Emparedados y helados
3,10441,55,3,1997-02-10 00:00:00,2,Ana Trujillo Emparedados y helados
4,10439,51,6,1997-02-07 00:00:00,3,Antonio Moreno Taquería
...,...,...,...,...,...,...
191,10252,76,4,1996-07-09 00:00:00,2,Ana Trujillo Emparedados y helados
192,10250,34,4,1996-07-08 00:00:00,2,Ana Trujillo Emparedados y helados
193,10251,84,3,1996-07-08 00:00:00,1,Alfreds Futterkiste
194,10249,81,6,1996-07-05 00:00:00,1,Alfreds Futterkiste


In [163]:
pd.read_sql(
    '''
    SELECT OrderDate , ShipperName FROM  (SELECT   Orders.*, Shippers.ShipperName FROM Orders
    join Shippers on Shippers.ShipperID = Orders.ShipperID order by OrderDate DESC )
    limit 1 offset 9
    ''',
    con,
)

,OrderDate,ShipperName
0,1997-02-03 00:00:00,Federal Shipping


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [113]:
pd.read_sql(
    '''
    with order_price as  (SELECT OrderDetails.*,  OrderDetails.Quantity * Products.Price as Price,  Products.ProductName  FROM OrderDetails join Products
                            on  OrderDetails.ProductID = Products.ProductID )
    select
    order_price.ProductName, order_price.Price
    from order_price
    inner join (select OrderID, sum(Price) as total_pice from order_price group by OrderID order by total_pice DESC limit 1) max_price
    on max_price.OrderID = order_price.OrderID
    ''',
    con,
)

,ProductName,Price
0,Sir Rodney's Marmalade,972.0
1,Côte de Blaye,10540.0
2,Camembert Pierrot,2380.0
3,Mozzarella di Giovanni,1461.6


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [114]:
pd.read_sql(
    '''
    with order_price as  (SELECT OrderDetails.*,  Products.Price, Products.ProductName  FROM OrderDetails join Products
                            on  OrderDetails.ProductID = Products.ProductID )
    select
    order_price.ProductName, order_price.Quantity
    from order_price
    inner join (select ProductID, sum(Quantity) as total_buy from order_price group by ProductID order by total_buy DESC limit 1) max_buy
    on max_buy.ProductID = order_price.ProductID
    ''',
    con,
)

,ProductName,Quantity
0,Gorgonzola Telino,20
1,Gorgonzola Telino,40
2,Gorgonzola Telino,15
3,Gorgonzola Telino,4
4,Gorgonzola Telino,25
5,Gorgonzola Telino,56
6,Gorgonzola Telino,30
7,Gorgonzola Telino,70
8,Gorgonzola Telino,20
9,Gorgonzola Telino,30


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [54]:
pd.read_sql(
    '''
    with order_price as  (SELECT OrderDetails.*, prod_sup.SupplierName, prod_sup.Country, prod_sup.ContactName ,prod_sup.Phone
                              FROM OrderDetails join
                            (select Products.ProductID, Suppliers.* from Products
                            join Suppliers on Products.SupplierID =  Suppliers.SupplierID ) prod_sup
                            on  OrderDetails.ProductID = prod_sup.ProductID )
    select
    order_price.SupplierName, order_price.Country, order_price.ContactName ,order_price.Phone,
      count(distinct(OrderID)) as total_buy from order_price group by SupplierName order by total_buy DESC limit 5
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone,total_buy
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755,37
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343,37
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448,33
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010,30
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323,27


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [121]:
pd.read_sql(
    '''
    with order_country as (select Orders.OrderID from Orders
    inner join (select Customers.CustomerID, Customers.Country from Customers where Country = "Brazil") need_cust on Orders.CustomerID = need_cust.CustomerID ),
    ored_prod  as (select OrderDetails.ProductID, OrderDetails.Quantity from OrderDetails
    inner join order_country on OrderDetails.OrderID = order_country.OrderID),
    product_cat as (select Products.*, Categories.CategoryName from Products join Categories on Categories.CategoryID = Products.CategoryID),
    cat_prod_price as ( select  product_cat.CategoryName, ored_prod.Quantity  * product_cat.Price as price_sum,
    product_cat.ProductID
    from ored_prod join product_cat on product_cat.ProductID = ored_prod.ProductID )
    select
    "Brazil" as Country,
    CategoryName,
    sum(price_sum) as cat_sum
    from cat_prod_price
    group by CategoryName
    order by cat_sum DESC limit 1
    ''',
    con,
)

,Country,CategoryName,cat_sum
0,Brazil,Beverages,13690.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [126]:
pd.read_sql(
    '''
    with order_country as (select Orders.OrderID from Orders
    inner join (select Customers.CustomerID, Customers.Country from Customers where Country = "USA") need_cust on Orders.CustomerID = need_cust.CustomerID ),
    ored_prod  as (select OrderDetails.ProductID, OrderDetails.Quantity, OrderDetails.OrderID from OrderDetails
    inner join order_country on OrderDetails.OrderID = order_country.OrderID),
    order_prod_price as ( select  ored_prod.Quantity  * Products.Price as price_sum, ored_prod.OrderID , Products.ProductID
    from ored_prod join Products on Products.ProductID = ored_prod.ProductID),
    order_total_price as ( select sum(price_sum) as order_sum from order_prod_price group by OrderID )
    select
    max(order_sum) - min(order_sum)
    from order_total_price
    ''',
    con,
)

,max(order_sum) - min(order_sum)
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [169]:
pd.read_sql(
    '''
    with most_yung as (SELECT * FROM Employees order by BirthDate DESC limit 3)
    select
    count(distinct(Orders.OrderID)),
    (FirstName || " " ||  LastName) as 'Full_Name'
    from Orders inner join most_yung
    on most_yung.EmployeeID = Orders.EmployeeID
    group by Orders.EmployeeID

    ''',
    con,
)

,count(distinct(Orders.OrderID)),Full_Name
0,29,Nancy Davolio
1,31,Janet Leverling
2,6,Anne Dodsworth


10. Сколько банок крабового мяса всего было заказано.

In [181]:
pd.read_sql(
    '''
    select ProductName,sum(Quantity)
    from OrderDetails inner join (
    SELECT ProductID, ProductName FROM Products
    where ProductName like "%crab%" ) crab_id on crab_id.ProductID = OrderDetails.ProductID
    ''',
    con,
)

,ProductName,sum(Quantity)
0,Boston Crab Meat,256
